In [152]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("./results/domain_adaptation_model", device='cuda')

In [153]:
from dotenv import load_dotenv
from os import environ as os_environ

from json import loads as json_loads

from groq import Groq

import telebot

import pandas as pd
import numpy as np

load_dotenv()

BOT_TOKEN = os_environ.get('BOT_TOKEN')
GROQ_API_KEY = os_environ.get("GROQ_API_KEY")

BOT = telebot.TeleBot(BOT_TOKEN)
CLIENT = Groq( api_key=GROQ_API_KEY )


In [154]:
df = pd.read_parquet('data/triples_corpus.parquet')

In [155]:
entities = {i: e for e, i in zip(df.subject.tolist() + df.object.tolist(), df.subjectId.tolist() + df.objectId.tolist())}
emb_ents = model.encode(list(entities.values()))  # el embedding está normalizado

In [162]:
authors_embs = model.encode(df.query("property=='wrote'").subject.tolist())

In [114]:
# Calcular la similitud coseno
cos_simi = emb_ents @ emb_q

# Obtener los índices de las 10 entidades con mayor similitud coseno
top_k_indices = np.argsort(cos_simi)[::-1]

# Recuperar las entidades correspondientes a estos índices
top_k_entities = [(list(entities.keys())[i],entities[list(entities.keys())[i]]) for i in top_k_indices]

In [115]:
ids = [i[0] for i in top_k_entities[:10]]

In [149]:
autb = model.encode("Susan H. Fisher")
aut = model.encode("Kudenko D.")

In [150]:
autb @ emb_q1, aut @ emb_q1

(0.031228611, 0.17037265)

In [151]:
 autb @ emb_q2, aut @ emb_q2

(0.20633332, 0.09790592)

In [124]:
df[df.subjectId.isin(ids) | df.objectId.isin(ids)]

,subject,property,object,subjectId,objectId
126,Herrera F.,wrote,Computing with Words in Decision support Syste...,1699069,16c6c16c98325e07bfcd099af2ea049fdcb758d3
191,Kudenko D.,wrote,Dynamic Generation of Dilemma-based Interactiv...,2380005,3fcbf761161a98cad1b36191ac3f3cc555b3c0ab
1030,Barber Heather,wrote,Dynamic Generation of Dilemma-based Interactiv...,144741684,3fcbf761161a98cad1b36191ac3f3cc555b3c0ab
1474,F. Koss,wrote,Intelligent Agents for Interactive Simulation ...,8590686,3fe78bb94a6683187cc2cab997fe40200694f9b3
2106,Computing with Words in Decision support Syste...,related with,An Approach to Computing With Words Based on C...,16c6c16c98325e07bfcd099af2ea049fdcb758d3,e9ddfeb230bd6e5553f1c5d68cb59610a84e72bd
2125,Dynamic Generation of Dilemma-based Interactiv...,related with,A social-psychological model for synthetic actors,3fcbf761161a98cad1b36191ac3f3cc555b3c0ab,e5fc3297133c2833bec9561f1e00c82facac7214
2290,Intelligent Agents for Interactive Simulation ...,related with,HCSM: a framework for behavior and scenario co...,3fe78bb94a6683187cc2cab997fe40200694f9b3,bea2b844f1d09ae154b6cb1c64282fa38ac70049
3457,Robyn L. Ball,published in,AAAI Conference on Artificial Intelligence,2054772900,2a163f1f-1cc2-3419-9a32-82f49889e8e6
3560,Ian Gent P.,published in,AAAI Conference on Artificial Intelligence,1726451,2a163f1f-1cc2-3419-9a32-82f49889e8e6
3572,Kudenko D.,published in,Artificial Intelligence and Interactive Digita...,2380005,4b86a22d-087f-3ba3-8b2a-f6b6c7fddd73


0          1751907
1         79187876
2          2220105
3        144359098
4          2254514
           ...    
1995       1720529
1996       1680452
1997     144004586
1998    2109090572
1999       2276812
Name: subjectId, Length: 2000, dtype: object

In [104]:
df.query('subjectId=="2651688"').iloc[1].object

'Annual Conference on Innovation and Technology in Computer Science Education'

In [ ]:
def request_recepcionist(user_request, system_addition = '', user_addition = ''):
    chat_completion = CLIENT.chat.completions.create(
        messages=[
            {
            "content": f"{personalities.roles['recepcionist']} {system_addition}",
            "role":"system"
            },
            {
                "role": "user",
                "content": f"{user_request} {user_addition}",
            }
        ],
        temperature=0.2,
        top_p=0.2,
        max_tokens=1000,
        response_format={"type": "json_object"},
        model="mixtral-8x7b-32768",
    )
    
    try:
        response = chat_completion.choices[0].message.content
        response_json = json_loads(response)
    except Exception as e:
        print(e)
        response_json = {'message':user_request, 'answer':'no_aplica'}
        
    return response_json


In [ ]:
@BOT.message_handler(commands=['start', 'hello'])
def send_welcome(message):
    print('holaaa!')
    BOT.reply_to(message, "Howdy, how are you doing?")
    
    
@BOT.message_handler(commands=['wtf'])
def send_welcome(message):
    BOT.reply_to(message, "No enteidno cpomo funcionas extooo")
    
    

@BOT.message_handler(func=lambda msg: True)
def echo_all(message):
    print(message.text, type(message))
    txt = message.text
    if 'help' in txt:
        BOT.reply_to(message, 'kk')
    else:
        BOT.reply_to(message, message.text)

In [ ]:
BOT.infinity_polling()